In [1]:
import pandas as pd
import numpy as np
from utils import read_file, tokenize_latex

In [2]:
import cPickle as pickle
eq_idx = pickle.load(open('data/numbered_embeddings/eq-idx.pkl', 'rb'))
eq_svd_embed = pickle.load(open('data/numbered_embeddings/eq-svd-embed.pkl', 'rb'))

In [3]:
eq_idx

['5.11.E7',
 '5.11.E4b',
 '5.11.E4e',
 '5.11.E9',
 '5.11.E15a',
 '5.11.E11',
 '5.11.E16c',
 '5.11.E18',
 '5.11.E4d',
 '5.11.E8',
 '5.11.E4c',
 '5.11.E1',
 '5.11.E6',
 '5.11.E19',
 '5.11.E17',
 '5.11.E10',
 '5.11.E16b',
 '5.11.E13',
 '5.11.E16a',
 '5.11.E14',
 '5.11.E15c',
 '5.11.E4g',
 '5.11.E5',
 '5.11.E2',
 '5.11.E15b',
 '5.11.E12',
 '5.11.E3',
 '5.11.E4f',
 '5.11.E4a',
 '5.9.E13',
 '5.9.E14',
 '5.9.E3',
 '5.9.E4',
 '5.9.E15',
 '5.9.E12',
 '5.9.E5',
 '5.9.E2',
 '5.9.E1',
 '5.9.E6',
 '5.9.E8',
 '5.9.E11',
 '5.9.E16',
 '5.9.E18',
 '5.9.E9',
 '5.9.E7',
 '5.9.E19',
 '5.9.E17',
 '5.9.E10',
 '5.4.E13b',
 '5.4.E15',
 '5.4.E3',
 '5.4.E20',
 '5.4.E4',
 '5.4.E1a',
 '5.4.E5',
 '5.4.E2',
 '5.4.E14',
 '5.4.E12b',
 '5.4.E8',
 '5.4.E1b',
 '5.4.E6',
 '5.4.E17',
 '5.4.E10',
 '5.4.E19',
 '5.4.E12a',
 '5.4.E18',
 '5.4.E11',
 '5.4.E16',
 '5.4.E7',
 '5.4.E9',
 '5.4.E13a',
 '5.16.E2',
 '5.16.E1',
 '5.20.E4',
 '5.20.E3',
 '5.20.E2',
 '5.20.E5',
 '5.20.E1',
 '5.12.E2',
 '5.12.E5',
 '5.12.E10',
 '5.12.E11',


In [4]:
import fnmatch
import os
def read_file_names_recursive(folder):
    matches = []
    for root, dirnames, filenames in os.walk(folder):
        for filename in fnmatch.filter(filenames, '*.tex'):
            matches.append(os.path.join(root, filename))
    return matches

In [5]:
fnames = read_file_names_recursive('data/numbered_embeddings/raw/')

In [6]:
def read_files(fnames):
    tex = {}
    for fname in fnames:
        f = open(fname).read()
        name = fname.split("/")[-1]
        name = name.strip(".tex")
        tex[name] = f
    return tex

In [7]:
tex = read_files(fnames)

In [8]:
eq_svd_embed.shape


(9884, 100)

In [9]:
from MulticoreTSNE import MulticoreTSNE as TSNE

tsne = TSNE(n_jobs=4)
Y = tsne.fit_transform(X)

NameError: name 'X' is not defined

In [10]:
tex_df = pd.DataFrame.from_dict(tex, orient="index")
tex_df = tex_df.reset_index()
tex_df.columns = ["eq_id", "tex"]

eq_idx_df = pd.DataFrame(eq_idx).reset_index()
eq_idx_df.columns = ["matrix_index", "eq_id"]

all_data = pd.merge(tex_df, eq_idx_df, on = "eq_id")

In [11]:
class Query:
    def __init__(self, term_matrix, docs, docs_dict, dictionary, cols, k):
        self.term_matrix = term_matrix
        self.docs = docs
        self.docs_dict = docs_dict
        self.dictionary = dictionary
        self.columns = cols
        self.k = k
        
    def _get_terms(self, vals):
        terms = []
        for i, v in vals:
            print i
            print v
            try:
                term = self.docs_dict[i]
                term["sim"] = v
                terms.append(term)
            except:
                print "couldn't find"
        return terms
    def query(self, query):
        idx = self.dictionary[query]["matrix_index"]
        vec = self.term_matrix[idx]
        
        idc, vals = self._vectorized_query(self.term_matrix, vec, self.k)
        
        zipped_vals= zip(idc, vals)
        vals = sorted(zipped_vals, key = lambda x: x[1])
        terms = self._get_terms(vals[:self.k])
        
        neighbors = {"neighbors":[{"Equation Number": {"data": t["eq_id"]}, 
                                   "Equation": {"data": t["tex"], "fmt": "math"}, 
                                   "Similarity": {"data": t["sim"]}} for t in terms]} if terms else {"neighbors": []}
        return neighbors
    
    def _vectorized_query(self, term_matrix, word_vector, k):
        dots = np.dot(term_matrix, word_vector)
        l2norms = np.sqrt(((term_matrix**2).sum(1)[:,None])*((word_vector**2).sum(0)))
        cosine_dists = 1 - (dots[:,None]/l2norms)
        cosine_dists = cosine_dists.reshape(dots.shape)
        idx = np.argpartition(cosine_dists, k)
        vals = cosine_dists[idx[:k]]
        return list(idx[:k].flatten()), list(vals.flatten())

In [12]:
dictionary = all_data.set_index("eq_id").to_dict(orient="index")
docs_dict = all_data.set_index("matrix_index").to_dict(orient="index")
docs = all_data["eq_id"].values
q = Query(eq_svd_embed, docs, docs_dict, dictionary, ["Equation Number", "Equation", "Similarity"], 20)

In [13]:
# example to test the query class
# q.query("34.4.E1")

In [14]:
all_data.head()

,eq_id,tex,matrix_index
0,4.35.E37,\[\mathop{\coth\/}\nolimits z=\frac{\mathop{\s...,6875
1,4.35.E30,\[\mathop{\cosh\/}\nolimits\!\left(3z\right)=-...,6874
2,4.28.E12,\[\mathop{\sec\/}\nolimits\!\left(iz\right)=\m...,7138
3,12.7.E7,"\[\mathop{U\/}\nolimits\!\left(n+\tfrac{1}{2},...",3914
4,12.7.E4,"\[\mathop{V\/}\nolimits\!\left(-\tfrac{1}{2},z...",3918


In [15]:
from mathviz_hopper.src.table import Table
t = Table(q, 8082)
t.print_ipython()

Bottle v0.13-dev server starting up (using MyWSGIRefServer())...
Listening on http://localhost:8082/
Hit Ctrl-C to quit.



127.0.0.1 - - [23/Dec/2017 10:54:14] "GET /settings HTTP/1.1" 200 66552
127.0.0.1 - - [23/Dec/2017 10:54:37] "GET /settings HTTP/1.1" 200 66552
